Les films les mieux notés et les caractéristiques qu’ils partagent.

In [146]:
# IMPORTS
import pandas as pd

df_title_ratings = pd.read_csv("./BD/title.ratings.tsv.gz", compression = 'gzip', sep = '\t')
df_title_basics = pd.read_csv("./BD/title.basics.tsv.gz", compression = 'gzip', sep = '\t')
df_tmdb_full = pd.read_csv("./BD/tmdb_full.csv")

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7466/445457019.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv("./BD/title.basics.tsv.gz", compression = 'gzip', sep = '\t')
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7466/445457019.py:6: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmdb_full = pd.read_csv("./BD/tmdb_full.csv")


In [147]:
# MERGES

df_film = df_title_basics[df_title_basics['titleType'] == 'movie']

df_1 = pd.merge(df_title_ratings,
                df_film,
                how = 'right',
                on = 'tconst') 

df_ratings = pd.merge(df_1,
                      df_tmdb_full,
                      how = 'outer',
                      left_on = 'tconst',
                      right_on = 'imdb_id') 

In [148]:
# Clean du Merge

df_ratings = df_ratings[~((df_ratings['vote_average'].isna()) & (df_ratings['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'] == 0))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'].isna()) & (df_ratings['averageRating'] == 0))]

df_ratings['vote_average'] = round(df_ratings['vote_average'], 1)

In [149]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_ratings[df_ratings['averageRating'] == df_ratings['vote_average']]
df_a_travailler_1 = df_ratings[~(df_ratings['averageRating'] == df_ratings['vote_average'])]


In [150]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[(df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0)]
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0))]

In [151]:
df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7466/2546417221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])


In [152]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] <= 1]
df_a_travailler_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] > 1]

In [153]:
df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
df_a_travailler_3['vote_exact'] = round(df_a_travailler_3['vote_exact'], 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7466/612399471.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7466/612399471.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler

In [154]:
df_a_garder_4 = df_a_travailler_3

In [155]:
# On crée la colonne vote_exact pour garder la moyenne des votes

df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_3['vote_exact'] = df_a_garder_3[['averageRating', 'vote_average']].mean(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7466/3653439724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7466/3653439724.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7466/36534397

In [156]:
# On concatène tous les tableaux conservés

df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])
df_final['arrondi_vote_exact'] = round(df_final['vote_exact'], 0)

In [191]:
meilleurs_films = df_final[df_final['vote_exact'] >= 7]
meilleurs_films_infos = meilleurs_films[['tconst', 'startYear', 'release_date', 'genres_x', 'genres_y', 'runtime', 'runtimeMinutes', 'budget', 'production_companies_name', 'vote_exact', 'spoken_languages']]

In [159]:
meilleurs_films_infos['total_genres'] = meilleurs_films_infos['genres_x'] + meilleurs_films_infos['genres_y']

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7466/3316027520.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meilleurs_films_infos['total_genres'] = meilleurs_films_infos['genres_x'] + meilleurs_films_infos['genres_y']


In [163]:
genres = meilleurs_films_infos['genres_x']
genres = genres.str.split(',' , expand = True)
df = genres[0].unique()
df2 = pd.DataFrame(df, columns = ['Genre'])
df2['count'] = ""

In [164]:
df2['Genre'] = df2['Genre'].astype(str)
meilleurs_films_infos['total_genres'] = meilleurs_films_infos['total_genres'].astype(str)


df2['count'] = df2['Genre'].apply(
    lambda genre: sum(genre in genres for genres in meilleurs_films_infos['total_genres'])
)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7466/2763424087.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meilleurs_films_infos['total_genres'] = meilleurs_films_infos['total_genres'].astype(str)


In [182]:
df2 = df2[~(df2['Genre'] == 'nan')]
df2 = df2[~(df2['Genre'] == '\\N')]

In [186]:
df2['Pourcentage apparition'] = round((df2['count']/df2['count'].sum())*100, 2)
df2.sort_values(by = 'count', ascending = False)

,Genre,count,Pourcentage apparition
0,Drama,20166,24.91
3,Documentary,13661,16.87
2,Comedy,8227,10.16
7,Romance,5269,6.51
18,Music,3496,4.32
9,Crime,3427,4.23
5,Action,3317,4.10
8,History,3097,3.83
17,Thriller,2895,3.58
6,Biography,2851,3.52
